In [1]:
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Step 1. Load Dataset
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=False)

# Step 2. Make Dataset Iterable
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
# Step 3. Create Model Class
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [3]:
n_iters = 3000
epochs = n_iters / (len(train_dataset) / batch_size)
input_dim = 784
output_dim = 10
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# output_dim = 1
lr_rate = 0.001

# Step 4. Instantiate Model Class
model = LogisticRegression(input_dim, output_dim)
# Step 5. Instantiate Loss Class
criterion = torch.nn.CrossEntropyLoss() # computes softmax and then the cross entropy
# [test] 만일 MSE을 LOSS 함수로 쓴다면???
# criterion = torch.nn.MSELoss()
# Step 6. Instantiate Optimizer Class
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [4]:
# Step 7. Train Model
iter = 0
for epoch in range(int(epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        # [test] 만일 MSE을 LOSS 함수로 쓴다면???
        # labels = labels.type(torch.FloatTensor)
        # outputs = outputs.type(torch.FloatTensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter+=1
        if iter%500==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for images, labels in test_loader:
                images = Variable(images.view(-1, 28*28))
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                # for gpu, bring the predicted and labels back to cpu fro python operations to work
                correct+= (predicted == labels).sum()
            accuracy = 100 * correct/total
            print("Iteration: {}. Loss: {}. Accuracy: {}.".format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.8824325799942017. Accuracy: 68.69999694824219.
Iteration: 1000. Loss: 1.5503674745559692. Accuracy: 77.62000274658203.
Iteration: 1500. Loss: 1.3944940567016602. Accuracy: 80.4000015258789.
Iteration: 2000. Loss: 1.2212333679199219. Accuracy: 81.83000183105469.
Iteration: 2500. Loss: 1.097227692604065. Accuracy: 82.73999786376953.
Iteration: 3000. Loss: 0.9613311290740967. Accuracy: 83.47000122070312.
